In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from datetime import datetime
import numpy as np


In [2]:
df_bundesland_all = pd.read_csv('df_bundesland_all.csv')

In [3]:
# Nach Bundesland sortieren
df_bundesland_all = df_bundesland_all.sort_values(by='Bundesland')

# Anzahl der Einträge pro Bundesland berechnen
bundesland_counts = df_bundesland_all['Bundesland'].value_counts().sort_index()

# Ergebnis anzeigen
print(bundesland_counts)

# Optional: Ausgabe in einem DataFrame speichern
df_bundesland_counts = bundesland_counts.reset_index()
df_bundesland_counts.columns = ['Bundesland', 'Anzahl Einträge']


Bundesland
Baden-Württemberg          57676
Bayern                     70309
Berlin                      8645
Brandenburg                15616
Bremen                      2993
Hamburg                     6468
Hessen                     35676
Mecklenburg-Vorpommern     11384
Niedersachsen              58934
Nordrhein-Westfalen       110549
Rheinland-Pfalz            28617
Saarland                    4226
Sachsen                    18276
Sachsen-Anhalt             11209
Schleswig-Holstein         18511
Thüringen                  14530
Name: count, dtype: int64


In [4]:
df_bundesland_all = df_bundesland_all[
    (df_bundesland_all['Bundesland'] != 'Bayern') &
    (df_bundesland_all['Bundesland'] != 'Baden-Württemberg') &
    (df_bundesland_all['Bundesland'] != 'Berlin') &
    (df_bundesland_all['Bundesland'] != 'Brandenburg') &
    (df_bundesland_all['Bundesland'] != 'Bremen') &
    (df_bundesland_all['Bundesland'] != 'Hamburg') &
    (df_bundesland_all['Bundesland'] != 'Hessen') &
    (df_bundesland_all['Bundesland'] != 'Mecklenburg-Vorpommern') &
    (df_bundesland_all['Bundesland'] != 'Niedersachsen') &
    (df_bundesland_all['Bundesland'] != 'Nordrhein-Westfalen') &
    (df_bundesland_all['Bundesland'] != 'Saarland') &
    (df_bundesland_all['Bundesland'] != 'Rheinland-Pfalz') &
    (df_bundesland_all['Bundesland'] != 'Sachsen') &
    (df_bundesland_all['Bundesland'] != 'Sachsen-Anhalt') &
    (df_bundesland_all['Bundesland'] != 'Schleswig-Holstein') &
    (df_bundesland_all['Bundesland'] != 'Thüringen')

]


In [5]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from datetime import datetime

# DataFrame df_bundesland_all verwenden
df_all = df_bundesland_all.copy()

# Zähler für die Gesamtzahl der Geokodierungen
total_count = 0

# Funktion zur Geokodierung basierend auf der ORTPLZ und der STRASSE
geolocator = Nominatim(user_agent="CapStone")

def get_coordinates(row, count):
    global total_count
    try:
        location = geolocator.geocode(f"{row['ORTPLZ']}, {row['STRASSE']}, Germany")
        if location:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            total_count += 1
            print(f"{total_count}: Geokodierung erfolgreich für ANO_SID: {row['ANO_SID']} - Bundesland: {row['Bundesland']} - ORTPLZ: {row['ORTPLZ']} - STRASSE: {row['STRASSE']} - Latitude: {location.latitude}, Longitude: {location.longitude} - Zeitstempel: {timestamp}")
            count += 1
            return location.latitude, location.longitude, count
        else:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            total_count += 1
            print(f"{total_count}: Keine Koordinaten gefunden für ANO_SID: {row['ANO_SID']} - Bundesland: {row['Bundesland']} - ORTPLZ: {row['ORTPLZ']} - STRASSE: {row['STRASSE']} - Zeitstempel: {timestamp}")
            count += 1
            return None, None, count
    except Exception as e:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        total_count += 1
        print(f"{total_count}: Fehler bei ANO_SID: {row['ANO_SID']} - Bundesland: {row['Bundesland']} - ORTPLZ: {row['ORTPLZ']} - STRASSE: {row['STRASSE']} - Fehler: {e} - Zeitstempel: {timestamp}")
        count += 1
        return None, None, count

def split_and_save_nrw(subset, part):
    count = 0
    latitude = []
    longitude = []
    for index, row in subset.iterrows():
        lat, lon, count = get_coordinates(row, count)
        latitude.append(lat)
        longitude.append(lon)
    
    subset['LATITUDE'] = latitude
    subset['LONGITUDE'] = longitude
    
    # Speichern des aktualisierten Subsets
    try:
        subset.to_csv(f'coord_subset_Nordrhein-Westfalen-{part}.csv', index=False)
        print(f'Subset für Nordrhein-Westfalen-{part} gespeichert als coord_subset_Nordrhein-Westfalen-{part}.csv')
    except Exception as e:
        print(f"Fehler beim Speichern des Subsets für Nordrhein-Westfalen-{part}: ANO_SID: {row['ANO_SID']} - ORTPLZ: {row['ORTPLZ']} - STRASSE: {row['STRASSE']} - Fehler: {e}")

# Gruppierung des DataFrames nach 'Bundesland'
grouped_bundesland = df_all.groupby('Bundesland')

# Geokodierung und Speichern der Subsets
for bundesland, subset in grouped_bundesland:
    if bundesland == 'Nordrhein-Westfalen':
        # Split into three parts
        subsets = np.array_split(subset, 3)
        # Only process and save the third subset
        split_and_save_nrw(subsets[2], 3)
    else:
        count = 0
        latitude = []
        longitude = []
        for index, row in subset.iterrows():
            lat, lon, count = get_coordinates(row, count)
            latitude.append(lat)
            longitude.append(lon)
        
        subset['LATITUDE'] = latitude
        subset['LONGITUDE'] = longitude
        
        # Speichern des aktualisierten Subsets
        try:
            subset.to_csv(f'coord_subset_{bundesland}.csv', index=False)
            print(f'Subset für Bundesland {bundesland} gespeichert als coord_subset_{bundesland}.csv')
        except Exception as e:
            print(f"Fehler beim Speichern des Subsets für Bundesland {bundesland}: ANO_SID: {row['ANO_SID']} - ORTPLZ: {row['ORTPLZ']} - STRASSE: {row['STRASSE']} - Fehler: {e}")
